In [1]:
import torch
import wandb
import sys
from configmypy import ConfigPipeline, YamlConfig, ArgparseConfig
from neuralop import get_model
from neuralop import Trainer
from neuralop.training import setup
from neuralop.datasets import load_darcy_pt
from neuralop.utils import get_wandb_api_key, count_params
from neuralop import LpLoss, H1Loss

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c105ErrorC2ENS_14SourceLocationESs
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import numpy as np
for i in np.arange(0.1, 1.0, 0.1):
    # Read the configuration
    config_name = 'default'
    pipe = ConfigPipeline([YamlConfig('./tfno_darcy_config.yaml', config_name='default', config_folder='./config'),
                          ])
    config = pipe.read_conf()
    config_name = pipe.steps[-1].config_name
    
    config.tfno2d.rank = i
    
    # Set-up distributed communication, if using
    device, is_logger = setup(config)
    
    # Make sure we only print information when needed
    config.verbose = config.verbose and is_logger

    #Print config to screen
    if config.verbose and is_logger:
        pipe.log()
        sys.stdout.flush()
    
    # Loading the Darcy flow training set in 32x32 resolution, test set in 32x32 and 64x64 resolutions
    train_loader, test_loaders, output_encoder = load_darcy_pt(
            config.data.folder, train_resolution=config.data.train_resolution, n_train=config.data.n_train, batch_size=config.data.batch_size, 
            positional_encoding=config.data.positional_encoding,
            test_resolutions=config.data.test_resolutions, n_tests=config.data.n_tests, test_batch_sizes=config.data.test_batch_sizes,
            encode_input=config.data.encode_input, encode_output=config.data.encode_output,
            )
    
    model = get_model(config)
    model = model.to(device)

    #Log parameter count
    if is_logger:
        n_params = count_params(model)

        if config.verbose:
            print(f'\nn_params: {n_params}')
            sys.stdout.flush()
    
    #Create the optimizer
    optimizer = torch.optim.Adam(model.parameters(), 
                                    lr=config.opt.learning_rate, 
                                    weight_decay=config.opt.weight_decay)

    if config.opt.scheduler == 'ReduceLROnPlateau':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=config.opt.gamma, patience=config.opt.scheduler_patience, mode='min')
    elif config.opt.scheduler == 'CosineAnnealingLR':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.opt.scheduler_T_max)
    elif config.opt.scheduler == 'StepLR':
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                                    step_size=config.opt.step_size,
                                                    gamma=config.opt.gamma)
    else:
        raise ValueError(f'Got {config.opt.scheduler=}')
    
    # Creating the losses
    l2loss = LpLoss(d=2, p=2)
    h1loss = H1Loss(d=2)
    if config.opt.training_loss == 'l2':
        train_loss = l2loss
    elif config.opt.training_loss == 'h1':
        train_loss = h1loss
    else:
        raise ValueError(f'Got training_loss={config.opt.training_loss} but expected one of ["l2", "h1"]')
    eval_losses={'h1': h1loss, 'l2': l2loss}
    
    if config.verbose and is_logger:
        print('\n### MODEL ###\n', model)
        print('\n### OPTIMIZER ###\n', optimizer)
        print('\n### SCHEDULER ###\n', scheduler)
        print('\n### LOSSES ###')
        print(f'\n * Train: {train_loss}')
        print(f'\n * Test: {eval_losses}')
        print(f'\n### Beginning Training...\n')
        sys.stdout.flush()
    
    trainer = Trainer(model, n_epochs=config.opt.n_epochs,
                      device=device,
                      mg_patching_levels=config.patching.levels,
                      mg_patching_padding=config.patching.padding,
                      mg_patching_stitching=config.patching.stitching,
                      wandb_log=config.wandb.log,
                      log_test_interval=config.wandb.log_test_interval,
                      log_output=False,
                      use_distributed=config.distributed.use_distributed,
                      verbose=config.verbose and is_logger)
    
    trainer.train(train_loader, test_loaders,
                  output_encoder,
                  model, 
                  optimizer,
                  scheduler, 
                  regularizer=False, 
                  training_loss=train_loss,
                  eval_losses=eval_losses)

###############################
#####    CONFIGURATION    #####
###############################

Steps:
------
 (1) YamlConfig with config_file=./tfno_darcy_config.yaml, config_name=default, config_folder=./config

-------------------------------

Configuration:
--------------

verbose=True
arch=tfno2d
distributed.use_distributed=False
tfno2d.lifting_channels=32
tfno2d.data_channels=3
tfno2d.n_modes_height=12
tfno2d.n_modes_width=12
tfno2d.hidden_channels=32
tfno2d.projection_channels=32
tfno2d.n_layers=4
tfno2d.domain_padding=None
tfno2d.domain_padding_mode=one-sided
tfno2d.fft_norm=forward
tfno2d.norm=None
tfno2d.skip=linear
tfno2d.implementation=factorized
tfno2d.separable=0
tfno2d.preactivation=0
tfno2d.use_mlp=1
tfno2d.mlp.expansion=0.5
tfno2d.mlp.dropout=0
tfno2d.factorization=tucker
tfno2d.rank=0.1
tfno2d.fixed_rank_modes=None
tfno2d.dropout=0.0
tfno2d.tensor_lasso_penalty=0.0
tfno2d.joint_factorization=False
opt.n_epochs=150
opt.learning_rate=0.005
opt.training_loss=h1
opt.weig

In [3]:
import numpy as np
import pandas as pd